In [2]:
import cv2
from fastai.core import *
from fastai.vision import *
from fastai.utils import show_install
from facenet_pytorch import MTCNN
from kgl_deepfake.nb_00 import *
from kgl_deepfake.nb_01a import *
from kgl_deepfake.nb_01b import *
from kgl_deepfake.nb_02a import *
from kgl_deepfake.nb_03 import *

In [3]:
#show_install()

### Data

In [5]:
SOURCE = Path('../data/train_sample_videos/')

In [6]:
f = get_files(SOURCE, extensions=['.json'])[0]
annots = pd.read_json(f).T
annots.reset_index(inplace=True)
annots.rename({'index':'fname'}, axis=1, inplace=True)
annots.head()

,fname,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [4]:
f = get_files(SOURCE, extensions=['.json'])[0]
annots = pd.read_json(f).T
annots.reset_index(inplace=True)
annots.rename({'index':'fname'}, axis=1, inplace=True)
annots.head()

,fname,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


#### Get face detector

In [11]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [12]:
detector = MTCNN(device=device, post_process=False)

#### Remove videos in which no faces are detected.  

In [9]:
fnames = [SOURCE/o for o in annots.fname]

In [13]:
hasface = get_has_face(fnames, detector)

In [14]:
annots_hasface = annots[np.array(hasface)]

#### Create `DataBunch`

In [15]:
src = VideoFaceList.from_df(df=annots_hasface, path=SOURCE, cols='fname').split_by_rand_pct()

In [16]:
get_transforms

<function fastai.vision.transform.get_transforms(do_flip: bool = True, flip_vert: bool = False, max_rotate: float = 10.0, max_zoom: float = 1.1, max_lighting: float = 0.2, max_warp: float = 0.2, p_affine: float = 0.75, p_lighting: float = 0.75, xtra_tfms: Union[Collection[fastai.vision.image.Transform], NoneType] = None) -> Collection[fastai.vision.image.Transform]>

In [17]:
bs, sz = 32, 128

In [22]:
%%time
data = (src.label_from_df('label').transform(get_transforms(), size=sz)
        .databunch(bs=bs, num_workers=0).normalize(imagenet_stats))

CPU times: user 16.2 s, sys: 2.46 s, total: 18.7 s
Wall time: 11.6 s


In [23]:
%%time
xb, yb = next(iter(data.train_dl))

CPU times: user 14.8 s, sys: 2.09 s, total: 16.9 s
Wall time: 10 s


### Model

In [24]:
model = MesoNet()

### Learner

In [25]:
learn = Learner(data, model, metrics=accuracy)

### Train

In [26]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: 